In [336]:
import re
import requests
from bs4 import BeautifulSoup
import time
from urllib import parse


# 사이드 메뉴 열 필요 없어서 일땐 빼놈. @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

# 여기 참고해서 작성해봤었음. https://brunch.co.kr/@jk-lab/18




# 지역별로 버튼 클릭할 필요없이 걍 다 담겨있네..
# from selenium import webdriver

# chromedriver_dir = '/home/nasanmaro/Desktop/projects/yapen/test/selenium_crawling_test/chromedriver'
# driver = webdriver.Chrome(chromedriver_dir)
# driver.get('http://www.yapen.co.kr')
# time.sleep(5)

# # 왼쪽 메뉴를 연다. 
# loca = driver.find_element_by_class_name('yapenMenu')
# loca.click()
# time.sleep(5)

# # 드라이버 인스턴스로부터 현제 메뉴 연상태의 페이지 소스 받아서 source에 넣는다. 
# source = driver.page_source

# # soup 객체로 만듬.
# soup = BeautifulSoup(source, 'lxml')



# 이것까지 하면 사이드 메뉴여는것 됬다. 
##########뒤에 아직따라안한 부분.################################################


# sido = driver.find_element_by_class_name('sido_area_box')
# li = sido.find_elements_by_tag_name('li')
# li[0].click()
# time.sleep(5)

# gugun = driver.find_element_by_class_name('gugun_area_box')
# guli = gugun.find_element_by_tag_name('li')
# guuli.click()
# time.sleep(5)


#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@









#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
# location_name_list 뽑는 과정 

def location_name_list_crawler():
    request = requests.get("http://www.yapen.co.kr")
    response = request.text
    soup = BeautifulSoup(response, 'lxml')

    left_menu=soup.select('div.locLayer')
    # 풀빌라, MD추천 제외 14지역중 7지역 만남김.
    selected_left_menu= left_menu[2:9]
    

    # 여기에 list 형태로 지역,지역고유번호/(세부지역,고유번호) 넣고싶다. 
    # location_name_list =[ [지역1 ,지역1고유번호 , [ (고유번호,세부지역),(고유번호2,세부지역2)...] ],
    #                       [지역2 ,지역2고유번호 , [ (고유번호,세부지역),(고유번호2,세부지역2)...] ],..
    location_name_list=list()

    for location in selected_left_menu:
        # 지역 이름 먼저 뽑음 
        location_name = location.select('div.titleStyle')[0].get_text(strip=True)
        location_name_sub_list=list()
        location_name_sub_list.append(location_name)

        li=location.select('li')

        #for문 돌면서 (고유번호 세부지역) 리스트에 담은뒤 location_list에 넣겠다.
        location_detail_list=[] 
        for location_detail in li:
            onclick_value = location_detail['onclick']  #regionMove('1.003021','금산/논산');
            
            split_right = onclick_value.split(',')[0]
            split_left = onclick_value.split(',')[1]

            sub_location_no = re.findall("'(.+)'",split_right)[0]
            sub_location_name = re.findall("'(.+)'",split_left)[0]

            location_detail_tuple = (sub_location_no,sub_location_name)

             # (고유번호/세부지역) 리스트에 담음
            location_detail_list.append(location_detail_tuple)
        
        # 지역 고유번호부터 정규표현식으로 뽑아내서 담음.(세부지역 고유번호의 소숫점 뒤 3자리)
        
        location_detail_no_for_search = location_detail_list[0][0] # '1.003021'
        location_detaol_no = re.findall(".(\d\d\d)",location_detail_no_for_search)[0]
        location_name_sub_list.append(location_detaol_no)
        
        # location_name_sub_list (고유번호 세부지역) 리스트를 넣음.
        location_name_sub_list.append(location_detail_list)
        
        # 상위 리스트에 넣음 
        location_name_list.append(location_name_sub_list)
        
    return location_name_list
    
    
# location_dict 뽑는 과정 끝
######################################################################################












#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
#이제 각 페이지 location 가서 해커톤때 쓴 pension crawler 돌리고 싶다.
# 메인페이지에서 기본정보 3개만 여러개 팬션에게서 가져왔던것.

# url = "http://www.yapen.co.kr/region?location=015&subLocation=1.015001"

def pension_crawler(location_no,sub_location_no):

    params = {
            'location':location_no,
             'subLocation':sub_location_no,
        }

    url = "http://www.yapen.co.kr/region?" + parse.urlencode(params)

    request = requests.get(url)
    response = request.text
    soup = BeautifulSoup(response, 'lxml')


    title_list=list()
    img_file_list=list()
    price_list=list()
    pldx_list=list()
    discount_rate_list=list()

    title_uls = soup.select('ul.dest-place-opt-fea')
    for ul in title_uls:
        li=ul.select('li')
        title_list.append(li[1].get_text())

    price_uls = soup.select('ul.dest-place-opt-cast')
    for ul in price_uls:
        li=ul.select('li')
        price_list.append(li[1].get_text())


    img_file_divs = soup.select('div.imgBox')
    for div in img_file_divs:
        img_file_list.append(div.select('img')[0]['src'])

        list1 = re.split('/', div.select('img')[0]['src'])
        pldx_list.append(int(list1[5]))


    dest_place_pics = soup.select('div.dest-place-pic')
    for dest_place_pic in dest_place_pics:
        # dest_place_pic에는 dic가 2개 or 1게 있는데  discount_rate가 있는 경우는 div가 2개이며
        # 길이가 5여서 이것으로 discount_rate있고 없고 를 비교한다. 
        if(len(dest_place_pic)==5):
            discount_rate_string =dest_place_pic.select('div')[0].get_text(strip=True)
            # %문자 정규표현식으로 빼줌.
            discount_rate_int = int(re.search('(\d*)',discount_rate_string).group())
            discount_rate_list.append(discount_rate_int)
        else:
            discount_rate_list.append(0)


    sub_locations_info_list = [title_list, price_list, img_file_list,pldx_list,discount_rate_list]
    
    return sub_locations_info_list
        

###########################################################################################







###########################################################################################
##province_name_list 로부터 지역명,고유번호 받아서 각 세부지역별로 pension_crawler 
## 해서 기본정보 3개씩 모으는 크롤러##

def location_crawler():
    
    location_info_list=list()
    
    location_name_list = location_name_list_crawler()
    # location_name_list =[ [지역1 ,지역1고유번호 , [ (고유번호,세부지역),(고유번호2,세부지역2)...] ],
    #                       [지역2 ,지역2고유번호 , [ (고유번호,세부지역),(고유번호2,세부지역2)...] ],..
    
    for location in location_name_list:
        location_name = location[0]
        location_no = location[1]
        sub_location_list = location[2]
        for sub_location in sub_location_list:
            print(sub_location)
            sub_location_no = sub_location[0]
            sub_location_name = sub_location[1]
            sub_locations_info_list = pension_crawler(location_no, sub_location_no)
            
            #기존에 pension모델 1차적으로 이름,가격,이미지로  만들던것에
            #location, sub_location 속성 추가해서 이곳에서 만들면 될듯하다.
            # 일단은 리스트 형태로 정보 6개 묶어서 저장해보겠음.
            #[[location,sub_location,title,pricem,img_file,pldx]....팬션 999까지 한 리스트에
            
            for i in range(len(sub_locations_info_list[0])):
                location_info_list.append([location_name,
                                           sub_location_name,
                                           sub_locations_info_list[0][i],# title
                                           sub_locations_info_list[1][i],# price
                                           sub_locations_info_list[2][i],# img_file
                                           sub_locations_info_list[3][i],# pldx
                                           sub_locations_info_list[4][i]
                                          ])
    
    return location_info_list
    
#########################################################################################################









#############################################################################################
################pension-detail-crawler 작성중.################################################
########
######## 팬션별 세부정보 크롤링.

# p테그 없는것 24140
# 픽업가능타이틀,디테일 없는경우, 이용주의사항 디테일만 없는경우 32928
# 픽업가능타이틀,디테일 없는경우, 27114
# 픽업가능 있는경우 20405

import json

params = {
    'ypIdx': 21438
}

url = "http://www.yapen.co.kr/details?" + parse.urlencode(params)

request = requests.get(url)
response = request.text
soup = BeautifulSoup(response, 'lxml')

# name 
name_root = soup.select('div.wrap_1000')
name = name_root[0].select('h3')[0].get_text()

# address 
table = soup.select('table.pensionTbl')
trs = table[0].select('tr')
tds = trs[0].select('td')
address = tds[0].get_text()

# check_in, check_out 
tds2 = trs[1].select('td')
check_in_out = tds2[0].select('span')
check_in = check_in_out[0].get_text()
check_out = check_in_out[1].get_text()

# pickup
tds3 = trs[2].select('td')
pickup = tds3[0].get_text(strip=True)

# room_num
td4 = trs[3].select('td')
number_tags = td4[0].select('span')
room_string = number_tags[0].get_text()
room_num = int(re.search('(\d*)',room_string).group())

# info
td5 = trs[4].select('td')
infos = td5[0].select('p')
info = ''
for one_info in infos:
    info = info + '\n' + one_info.get_text() + '\n'

# theme 
td6 = trs[5].select('td')
lis = td6[0].select('li')
theme_list = list()
for li in lis:
    theme_list.append(li.get_text() )
theme = json.dumps(theme_list)


room_name_list = list()
image_table = soup.select('div.roomImageLists')
image_table_lis = image_table[0].select('li')
for index,image_table_li in enumerate(image_table_lis):
    if 0<index<room_num+1:
        room_name_list.append(image_table_li.get_text())
        

        
        
# pension_detail 페이지 하단 추가정보들 . 

# print(detailDiv)
detailDiv = soup.select('div.detailDiv')[0]
detailsPensionInfoTitle = detailDiv.select('div.detailsPensionInfoTitle')

pension_detail_bellow_dict = dict()

# print(detailsPensionInfoTitle)

for one_title in detailsPensionInfoTitle:
    next_detail_div = one_title.findNext('div')
    next_detail=''
    for p in next_detail_div.select('p'):
        next_detail= next_detail + p.get_text() +'\n'
    # p 테그없고 바로 text쓴경우에는 검출안된다.....

           
    pension_detail_bellow_dict[one_title.get_text(strip=True)] = next_detail

# print(pension_detail_bellow_dict)

# print(pension_detail_bellow_dict.get('입실 / 퇴실시간'))
# print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
# print(pension_detail_bellow_dict.get('픽업가능'))
# print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
# print(pension_detail_bellow_dict.get('펜션소개 및 인사말'))
# print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
# print(pension_detail_bellow_dict.get('이용 주의사항'))




check_in_out_detail = ""
pickup_detail = ""
gretting = ""
precautions = "" 


for key, value in pension_detail_bellow_dict.items():
    if key == '입실 / 퇴실시간':
        check_in_out_detail = value
    elif key == '픽업가능':
        pickup_detail = value
    elif key == '펜션소개 및 인사말':
        gretting = value
    elif key == '이용 주의사항':
        precautions =  value
        


        
        
print(name)
print(address)
print(check_in)
print(check_out)
print(room_num)
print(info_result)
print(theme)

print(check_in_out_detail)
print(pickup_detail) 
print(gretting)
print(precautions) 
        
        


# address = 
# check_in =
# check_out =
# pickup = 
# room_num = 
# info =
# theme = 
# coordinate = 

# # pension-detail 페이지 하단부 이용안내 부분.
# check_in_out_detail = 
# pickup_detail = 
# gretting = 
# precautions = 



#  해커톤때 모델저장하는것 크롤러에서 다해버렸는데 추후에 여기 참고하면 좋을듯.


#  params = {
#         'ypIdx': ypIdx
#     }

#     url = "http://www.yapen.co.kr/details?" + parse.urlencode(params)

#     request = requests.get(url)
#     response = request.text
#     soup = BeautifulSoup(response, 'lxml')

#     name_root = soup.select('div.wrap_1000')
#     name = name_root[0].select('h3')[0].get_text()

#     table = soup.select('table.pensionTbl')
#     trs = table[0].select('tr')
#     tds = trs[0].select('td')
#     address = tds[0].get_text()

#     tds2 = trs[1].select('td')
#     check_in_out = tds2[0].select('span')
#     check_in = check_in_out[0].get_text()
#     check_out = check_in_out[1].get_text()

#     td3 = trs[3].select('td')
#     number_tags = td3[0].select('span')
#     room = number_tags[0].get_text()

#     td4 = trs[4].select('td')
#     infos = td4[0].select('p')
#     info_result = ''
#     for info in infos:
#         info_result = info_result + '\n' + info.get_text() + '\n'

#     td5 = trs[5].select('td')
#     lis = td5[0].select('li')
#     theme_result = ''
#     for li in lis:
#         theme_result = theme_result + '\n' + li.get_text() + '\n'




#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
#@@@@@@@@@@@@@@@@@@@@@실행창 @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

# province_name_list = province_name_list_crawler()
# print(province_name_list)

# test = location_name_list_crawler()
# print(test)

# test = pension_crawler('015','1.015001')
# print(test)


# location_info_list = location_crawler()
# print(location_info_list)





가평 휴성펜션

지번 : 경기 가평군 북면 적목리 553-22 
도로명 : 경기 가평군 북면 가화로 3169-23                                

오후 3시 00분 이후
오전 11시 00분 이전
9

★2018년 수영장 오픈



※ 2018년 5월 1일 부터 수영장 이용 가능합니다!

· 운영시간 : 오후2시~9시

· 수영장 크기 : 가로 5.2m / 세로 10m / 수심1~1.2m

· 튜브나 놀이기구는 비치하고 있지 않습니다.

· 콤프레샤를 비치해놓고 있어 튜브나 공에 바람넣기는 수월합니다.

· 음식물 반입금지, 음주 후 입수금지, 다이빙 절대 금지

객실구조에 대한 안내 



전 객실 복층형태이며, 1층: 거실형온돌+주방1+화장실1+테라스 / 복층: 침대형원룸+주방1+화장실1+테라스로 구성되어 있습니다 

* 바비큐장은 2층 개별테라스에서 이용가능합니다. 

["\uc2a4\ud30c/\uc6d4\ud480", "\uc218\uc601\uc7a5", "\uac1c\ubcc4\ubc14\ube44\ud050", "\ub178\ub798\ubc29", "\ubcf5\uce35", "IPTV/WiFi", "\uacc4\uace1\uc778\uc811", "\ubc14\ube44\ud050\uc7a5"]
22시 이후 입실하실 경우 펜션으로 미리 연락 부탁드립니다.
객실 정리 후 퇴실점검을 받으셔야 합니다. 점검 후 객실키를 반납해 주세요.

가평터미널 인근 조은마트에서 픽업 요청 후 1인 기준 15,000원 정도 장을 보시면 펜션까지 픽업이 가능합니다.(왕복 픽업 가능) 







빛나는 별과 아름다운 밤핮늘 계곡의 노래가 함께하는 곳 EXTERIOR



예약 인원에서 인원이 추가되는 경우 펜션에 미리 연락을 주시기 바랍니다.
최대 인원 초과 시 입실이 불가능할 수 있으며, 해당 사유로 환불 받을 수 없습니다.
반려동물 입실 가능 펜션 외에 반려동물 동반 시 입실이 거부될 수 있으며, 해당 사유로 환불 받을 수 없습니다.

In [262]:
kkkk="""
    <div>
      <div class="detailsPensionInfoTitle">
        입실 / 퇴실시간
      </div>
      <div style="font-size:12px; line-height:180%;">
        <div class="timeInfo" style="padding:15px 16px; 10px;">
          <span class="checkin" style="color: #f9f9f9;padding: 5px;background: #70b2e9;font-size: 11px;letter-spacing: -1px;border-radius: 5px;margin-top: -2px;">
            입실
          </span>
          <font style="font-weight:bold; color:#FF6559; font-size: 15px;">
            15:00이후
          </font>
          <span class="checkin" style="color: #f9f9f9;padding: 5px;background: #70b2e9;font-size: 11px;letter-spacing: -1px;border-radius: 5px;margin-top: -2px;">
            퇴실
          </span>
          <font style="font-weight:bold; color:#FF6559; font-size: 15px;">
            11:00이전
          </font>
        </div>
        <div class="top_ex" style="margin-left: 18px; margin-bottom: 8px;">
          <p>
            22시 이후 입실하실 경우 펜션으로 미리 연락 부탁드립니다. 객실 깨끗이 정리 후 키 반납해주시길 바랍니다.
          </p>
        </div>
      </div>
    </div>
"""

In [263]:
sibling_soup2 = BeautifulSoup(kkkk)

/home/nasanmaro/.local/share/virtualenvs/selenium_crawling_test-TzOD7ppC/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/nasanmaro/.pyenv/versions/3.6.5/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [270]:
a = sibling_soup2.select_one('div.detailsPensionInfoTitle').findNext('div')
print(type(a))
print(a)

<class 'bs4.element.Tag'>
<div style="font-size:12px; line-height:180%;">
<div class="timeInfo" style="padding:15px 16px; 10px;">
<span class="checkin" style="color: #f9f9f9;padding: 5px;background: #70b2e9;font-size: 11px;letter-spacing: -1px;border-radius: 5px;margin-top: -2px;">
            입실
          </span>
<font style="font-weight:bold; color:#FF6559; font-size: 15px;">
            15:00이후
          </font>
<span class="checkin" style="color: #f9f9f9;padding: 5px;background: #70b2e9;font-size: 11px;letter-spacing: -1px;border-radius: 5px;margin-top: -2px;">
            퇴실
          </span>
<font style="font-weight:bold; color:#FF6559; font-size: 15px;">
            11:00이전
          </font>
</div>
<div class="top_ex" style="margin-left: 18px; margin-bottom: 8px;">
<p>
            22시 이후 입실하실 경우 펜션으로 미리 연락 부탁드립니다. 객실 깨끗이 정리 후 키 반납해주시길 바랍니다.
          </p>
</div>
</div>


In [ ]:
6

In [ ]:

#  province_dict  뽑는 과정인데 list로 바꾼후 여기다 일단 저장해 놓음. 나중에 필요하면 가져다 써라.




#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
# province_dict 뽑는 과정 

def province_dict_crawler():
    request = requests.get("http://www.yapen.co.kr")
    response = request.text
    soup = BeautifulSoup(response, 'lxml')

    left_menu=soup.select('div.locLayer')
    # 풀빌라, MD추천 제외 14지역중 7지역 만남김.
    selected_left_menu= left_menu[2:9]
    # print(selected_left_menu)

    # 여기에 dict 형태로 지역/(세부지역,고유번호) 넣고싶다. 
    # location_dict = {'지역':[(고유번호,세부지역),(고유번호2,세부지역2),(고유번호3,세부지역3)]}
    location_dict=dict()

    for location in selected_left_menu:
        # 지역 이름 먼저 뽑음 
        province_title = location.select('div.titleStyle')[0].get_text(strip=True)
    #     print(province_title)

        li=location.select('li')

        #for문 돌면서 (고유번호 세부지역) 리스트에 담은뒤 province_dict에 넣겠다.
        location_detail_list=[] 
        for location_detail in li:
            onclick_value = location_detail['onclick']  #regionMove('1.003021','금산/논산');  
            location_detail_tuple = tuple(re.findall("'(.+)'",onclick_value))

             # (고유번호 세부지역) 리스트에 담음
            location_detail_list.append(location_detail_tuple)

        # location_dict에 지역과 함께 (고유번호 세부지역) 리스트에 넣음.
        location_dict[province_title]= location_detail_list
        
    return location_dict
    
    
# province_dict 뽑는 과정 끝
######################################################################################